In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

%matplotlib inline

In [ ]:
sns.set_style("white")

In [ ]:
# Display figures at a reasonable default size.
mpl.rcParams['figure.figsize'] = (6, 4)

# Disable top and right spines.
mpl.rcParams['axes.spines.top'] = False
mpl.rcParams['axes.spines.right'] = False
    
# Display and save figures at higher resolution for presentations and manuscripts.
mpl.rcParams['savefig.dpi'] = 300
mpl.rcParams['figure.dpi'] = 300

# Display text at sizes large enough for presentations and manuscripts.
mpl.rcParams['font.weight'] = "normal"
mpl.rcParams['axes.labelweight'] = "normal"
mpl.rcParams['font.size'] = 14
mpl.rcParams['axes.labelsize'] = 14
mpl.rcParams['legend.fontsize'] = 12
mpl.rcParams['xtick.labelsize'] = 14
mpl.rcParams['ytick.labelsize'] = 14

## Load data

Load exhaustive grid search data. For each possible embedding method, corresponding method parameters, and HDBSCAN distance threshold, we produced an embedding for training and validation data (using 2-fold validation with 3 repeats for N=6 cross-validation iterations per parameter combination), assigned clusters to each embedding, and evaluated how well all pairs of strains in the data were assigned to the same or different cluster compared to predetermined clade assignments.

In [ ]:
grid = pd.read_csv(snakemake.input.table, sep="\t")

In [ ]:
grid.head()

In [ ]:
list(grid.columns)

## Identify the distance threshold that maximized the training accuracy

We want to identify the distance threshold for HDBSCAN cluster assignment that produces the most accurate clusters (based on known clade assignments) across each method. To find this optimal threshold, we calculate the mean training MCC value across all cross-validation iterations for a given method and set of method parameters and find the threshold that maximizes this mean across each method.

In [ ]:
mean_training_mcc_by_method = grid.groupby(["method", "distance_threshold"])["training_mcc"].mean().reset_index()

In [ ]:
mean_training_mcc_by_method

In [ ]:
facet_grid = sns.relplot(
    data=grid,
    x="distance_threshold",
    y="training_mcc",
    col="method",
    col_wrap=2,
    alpha=0.5,
)

for ax in facet_grid.axes.flatten():
    ax.set_ylim(0, 1)
    
plt.tight_layout()
plt.savefig(snakemake.output.mcc_by_method_and_distance_threshold)

Calculate the mean training MCC for each method and distance threshold across all cross-validation iterations. The maximum mean training MCC per method identifies the optimal distance threshold for that method.

In [ ]:
training_mcc = grid.groupby(["method", "distance_threshold"]).aggregate(
    {"training_mcc": ["mean", "std"]}
).reset_index().droplevel(1, axis=1).set_axis(
    ["method", "distance_threshold", "training_mcc_mean", "training_mcc_stddev"],
    axis="columns"
)

In [ ]:
training_mcc

In [ ]:
max_training_mcc = training_mcc.sort_values([
    "method",
    "training_mcc_mean"
], ascending=False).groupby(
    "method",
    sort=False
).first().reset_index()

In [ ]:
max_training_mcc

In [ ]:
if "perplexity" in grid.columns:
    facet_grid = sns.relplot(
        data=grid.query("method == 't-sne'"),
        x="distance_threshold",
        y="training_mcc",
        col="learning_rate",
        row="perplexity",
        alpha=0.75,
    )

    for ax in facet_grid.axes.flatten():
        ax.set_ylim(0, 1)

In [ ]:
if "min_dist" in grid.columns:
    facet_grid = sns.relplot(
        data=grid.query("method == 'umap'"),
        x="distance_threshold",
        y="training_mcc",
        col="n_neighbors",
        row="min_dist",
        alpha=0.75,
    )

    for ax in facet_grid.axes.flatten():
        ax.set_ylim(0, 1)

### PCA

In [ ]:
pca_max_training_mcc = max_training_mcc.query("method == 'pca'")

In [ ]:
pca_max_training_mcc

In [ ]:
try:
    pca_max_training_mcc.to_csv(
        snakemake.output.pca_parameters,
        index=False,
    )
except:
    print("does not exist")

### MDS

In [ ]:
mds_max_training_mcc = max_training_mcc.query("method == 'mds'")

In [ ]:
mds_max_training_mcc

In [ ]:
try:
    mds_max_training_mcc.to_csv(
        snakemake.output.mds_parameters,
        index=False,
    )
except:
    print("does not exist")

## Identify optimal method parameter values

Given the distance threshold that maximizes the training MCC above (threshold=4 for t-SNE and threshold=2 for UMAP), we next identify the combination of method parameters that maximizes the validation MCC for that distance threshold.

### t-SNE

In [ ]:
tsne_max_training_mcc = max_training_mcc.query("method == 't-sne'")

In [ ]:
tsne_grid = tsne_max_training_mcc.merge(
    grid,
    on=["method", "distance_threshold"]
)

In [ ]:
tsne_grid.head()

In [ ]:
try:
    tsne_mean_validation_mcc = tsne_grid.groupby([
        "method",
        "distance_threshold",
        "perplexity",
        "learning_rate"
    ])["validation_mcc"].mean().reset_index().sort_values(
        "validation_mcc",
        ascending=False
    )
except:
    tsne_mean_validation_mcc = tsne_grid.groupby([
        "method",
        "distance_threshold"
    ])["validation_mcc"].mean().reset_index().sort_values(
        "validation_mcc",
        ascending=False
    )

In [ ]:
tsne_mean_validation_mcc

In [ ]:
tsne_max_validation_mcc = tsne_mean_validation_mcc.sort_values("validation_mcc", ascending=False).head(1)

In [ ]:
tsne_max_validation_mcc

In [ ]:
try:
    tsne_max_validation_mcc.to_csv(
        snakemake.output.tsne_parameters,
        index=False,
    )
except:
    print("not a parameter")

In [ ]:
try: 
    facet_grid = sns.catplot(
        data=tsne_grid,
        x="perplexity",
        y="validation_mcc",
        hue="learning_rate",
        dodge=True,
        alpha=0.75,
    )
    
    for ax in facet_grid.axes.flatten():
        ax.set_ylim(0, 1)

    plt.savefig(snakemake.output.mcc_by_tsne_parameters)
except:
    print("no need for params")
    


### UMAP

In [ ]:
umap_max_training_mcc = max_training_mcc.query("method == 'umap'")

In [ ]:
umap_grid = umap_max_training_mcc.merge(
    grid,
    on=["method", "distance_threshold"]
)

In [ ]:
umap_grid.head()

In [ ]:
try:
    umap_mean_validation_mcc = umap_grid.groupby([
        "method",
        "distance_threshold",
        "min_dist",
        "n_neighbors"
    ])["validation_mcc"].mean().reset_index().sort_values(
        "validation_mcc",
        ascending=False
    )
except:
    umap_mean_validation_mcc = umap_grid.groupby([
        "method",
        "distance_threshold"
    ])["validation_mcc"].mean().reset_index().sort_values(
        "validation_mcc",
        ascending=False
    )

In [ ]:
umap_mean_validation_mcc

In [ ]:
umap_max_validation_mcc = umap_mean_validation_mcc.sort_values("validation_mcc", ascending=False).head(1)

In [ ]:
umap_max_validation_mcc

In [ ]:
try:
    umap_max_validation_mcc.to_csv(
        snakemake.output.umap_parameters,
        index=False,
    )
except:
    print("not a parameter")

In [ ]:
try:
    facet_grid = sns.catplot(
        data=umap_grid,
        x="min_dist",
        y="validation_mcc",
        hue="n_neighbors",
        dodge=True,
        alpha=0.75,
    )
    for ax in facet_grid.axes.flatten():
        ax.set_ylim(0, 1)

    plt.savefig(snakemake.output.mcc_by_umap_parameters)

except:
    print("no need for params")

